# Precios en tiempos de cólera

#### El salario cubano frente al costo de la canasta básica en MiPymes minoristas

En Cuba, el **precio** es sinónimo de decisión: determina qué puede comprar una persona con su salario en un mercado donde los precios suben sin límite y coexisten **dos tasas de cambio**. Entonces surge la pregunta: **¿Puede un cubano de la clase trabajadora adquirir los alimentos básicos?**

## Introducción

## Fuentes de datos

Para el desarrollo del análisis se realizó la recopilación de información de **30 MiPymes minorista**, considerando un conjunto mínimo de 10 productos de consumo básico (alimentos). A partir de esta información se obtuvieron precios reales del mercado local, como fuente principal.

Como complemento, se utilizaron fuentes de información externas relacionadas con el **salario mensual** del trabajador cubano y las **tasas de cambio** vigentes, con el objetivo de apreciar los precios y permitir comparaciones económicas. También, se realizó la extracción de precios de productos en páginas web que operan en divisa.

### Bibliotecas:

In [3]:
from m_tools import *
from m_normalize import *
from m_charts import *
from m_dataframe import Dataframe

In [ ]:
df_pymes = Dataframe().from_json("data/mipymes.json")

In [4]:
print(f'Cantidad de MiPymes: {data_count}')

# =============================================

print(f'Cantidad de MiPymes: {df_pymes.len()}')

In [5]:
print("Old: ", CountProducts2())

# ===================================

df = Dataframe().from_json("data/mipymes.json")
df = df.add_col(prod_count = [len(v) for v in df["products"]])
dfc = df.del_col("address","products") # "id","currency","town",
print("NEW: ")
print( df.get_table( column_limit=25))

## Canasta:

In [ ]:
Basket('basket.json')


df_unit = Dataframe(df_pymes.flatify("products", ["category", "count"]).sort("count").group_apply("category", "count", Moda))
print( df_unit)
df_unit.to_json('df_basket.json', func=lambda b: {k:v[0] for k,v in b.items()}, ensure_ascii=False, indent=2)

## Chart 1: Precio promedio por producto

In [ ]:
ChartAverageProducts()

# =============================

df_x_prod = (
    df_pymes
    .flatify("products", ["category", "records"])
    .explode("records", parent_keys=["category"], columns=["price"])
    .filter(lambda r: r["category"] != "huevos") 
    )

df_promed_x_prod = (
    df_x_prod
    .group_apply("category", "price", lambda x: sum(x)/len(x), as_new_row="avg_price")
    .sort("avg_price")
)

plt.subplots()
plt.bar(df_promed_x_prod["category"], df_promed_x_prod["avg_price"])
plt.title('Precio promedio x producto')
plt.xlabel('Productos')
plt.ylabel('Precios (CUP)')
plt.xticks(rotation=40)
plt.show()

## Chart 2: Variabilidad de precios

In [ ]:
ChartVariabilityPrices()

# ======================================

df_violin = df_x_prod.group_by("category", "price", as_new_row="avg_price")

plt.subplots()
plt.violinplot(df_violin["avg_price"], showmeans=True, showextrema=True)
plt.title("Distribución de precios por productos para gays")
plt.xlabel("Productos")
plt.xticks(range(1, len(df_violin["category"])+1), df_violin["category"], rotation=45)
plt.ylabel("Precio (CUP)")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

## Chart 3: Comparación de Canasta/Salario

In [ ]:
ChartCostBasket()

# ================================================

# Hasta aqui me dió pereza continuar, igual se va entendiendo..

In [ ]:
ChartCostBasket2()

## Char 4: Porcentaje del salario

In [ ]:
ChartPercetSalary()

## Chart 5: Días necesarios

In [ ]:
ChartNecesaryDays(30)